# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from pprint import pprint
from datetime import datetime

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Set Global Params

In [4]:
# Random state int:
random_state = 42

# n-estimators 
n_estimators = 5000

## Read the CSV and Perform Basic Data Cleaning

In [5]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

In [6]:
print(f"Column names : {df.columns.values}") # view column names

Column names : ['loan_amnt' 'int_rate' 'installment' 'home_ownership' 'annual_inc'
 'verification_status' 'issue_d' 'loan_status' 'pymnt_plan' 'dti'
 'delinq_2yrs' 'inq_last_6mths' 'open_acc' 'pub_rec' 'revol_bal'
 'total_acc' 'initial_list_status' 'out_prncp' 'out_prncp_inv'
 'total_pymnt' 'total_pymnt_inv' 'total_rec_prncp' 'total_rec_int'
 'total_rec_late_fee' 'recoveries' 'collection_recovery_fee'
 'last_pymnt_amnt' 'next_pymnt_d' 'collections_12_mths_ex_med'
 'policy_code' 'application_type' 'acc_now_delinq' 'tot_coll_amt'
 'tot_cur_bal' 'open_acc_6m' 'open_act_il' 'open_il_12m' 'open_il_24m'
 'mths_since_rcnt_il' 'total_bal_il' 'il_util' 'open_rv_12m' 'open_rv_24m'
 'max_bal_bc' 'all_util' 'total_rev_hi_lim' 'inq_fi' 'total_cu_tl'
 'inq_last_12m' 'acc_open_past_24mths' 'avg_cur_bal' 'bc_open_to_buy'
 'bc_util' 'chargeoff_within_12_mths' 'delinq_amnt' 'mo_sin_old_il_acct'
 'mo_sin_old_rev_tl_op' 'mo_sin_rcnt_rev_tl_op' 'mo_sin_rcnt_tl'
 'mort_acc' 'mths_since_recent_bc' 'mths_sinc

In [7]:
# Encode non-numerical data into binary data

# View non-numerical data types
df_data_types = df.dtypes.to_frame()
cols_w_unsupported_data_types = list(df_data_types[df_data_types.iloc[:,0] != 'float64'].index)
df_binary_encoded = pd.get_dummies(df, columns=cols_w_unsupported_data_types)

# Check df data types after encoding
print(f"Data types : {Counter(list(df_binary_encoded.dtypes.to_frame().iloc[:,0]))}")

Data types : Counter({dtype('float64'): 76, dtype('uint8'): 21})


## Split the Data into Training and Testing

In [8]:
# Create our features
X = df_binary_encoded.drop(columns=['loan_status_high_risk','loan_status_low_risk'])

# Create our target
y = df_binary_encoded['loan_status_high_risk']

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [10]:
# Check the balance of our target values
print(f"Balance of targer values: \n0: {Counter(y)[0]} | 1: {Counter(y)[1]}")

Balance of targer values: 
0: 68470 | 1: 347


In [11]:
from sklearn.model_selection import train_test_split

# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state) 

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [12]:
from sklearn.preprocessing import StandardScaler

# Create the StandardScaler instance
scaler = StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [14]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
print(f"Balance of targer values: \n0: {Counter(y_train)[0]} | 1: {Counter(y_train)[1]}")

Balance of targer values: 
0: 51369 | 1: 243


## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [16]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

startime = datetime.now() # start timer

brf = BalancedRandomForestClassifier(n_estimators=n_estimators,
                                     random_state=random_state)

brf_model = brf.fit(X_train_scaled, y_train)
brf_y_pred = brf_model.predict(X_test_scaled)

print(f"BalancedRandomForestClassifier completed in {str(datetime.now()-startime)}")

BalancedRandomForestClassifier completed in 0:00:56.710838


In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

print(f"Balanced Accuracy Score = {round(balanced_accuracy_score(y_test, brf_y_pred),3)}")

Balanced Accuracy Score = 0.787


In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(y_test, brf_y_pred), columns=['Pred-0','Pred-1'], index=['0','1'])

,Pred-0,Pred-1
0,15556,1545
1,35,69


In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, brf_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.91      0.66      0.95      0.78      0.62     17101
          1       0.04      0.66      0.91      0.08      0.78      0.59       104

avg / total       0.99      0.91      0.66      0.95      0.78      0.62     17205



In [20]:
# List the top 10 features sorted in descending order by feature importance
bfr_feat_imp_li = sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)
bfr_feat_imp_df = pd.DataFrame([(feat[1], feat[0]) for feat in bfr_feat_imp_li], columns=['feature','score']).set_index('feature')
bfr_feat_imp_df['pct_of_whole'] = (bfr_feat_imp_df['score'] / bfr_feat_imp_df['score'].sum()) * 100
bfr_feat_imp_df

,score,pct_of_whole
feature,,
total_rec_prncp,0.073643,7.364278
total_rec_int,0.060750,6.075050
last_pymnt_amnt,0.059764,5.976375
total_pymnt,0.059269,5.926946
total_pymnt_inv,0.058683,5.868259
...,...,...
num_tl_120dpd_2m,0.000000,0.000000
hardship_flag_N,0.000000,0.000000
debt_settlement_flag_N,0.000000,0.000000


### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=n_estimators, 
                             random_state=random_state)

eec_model = eec.fit(X_train_scaled, y_train)
eec_y_pred = eec_model.predict(X_test_scaled)

In [ ]:
# Calculated the balanced accuracy score
print(f"Balanced Accuracy Score = {round(balanced_accuracy_score(y_test, eec_y_pred),3)}")

In [ ]:
# Display the confusion matrix
pd.DataFrame(confusion_matrix(y_test, eec_y_pred), columns=['Pred-0','Pred-1'], index=['0','1'])

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, eec_y_pred))

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.